In [ ]:
import cv2
import time
from ultralytics import YOLO

emotion_classes = ['Angry','Boring','Disgust','Fear','Happy','Neutral','Sad','Stress','Suprise']
model = YOLO("D:/RuhunaNew/Academic/Research/Facial_Recog_Repo/Group_50_Repo/Final_Best_Codes/Models/yolo11n-cls.pt")

cap = cv2.VideoCapture(0)
inference_interval = 5  # seconds
last_inference_time = 0
last_label = "No emotion detected"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    if current_time - last_inference_time >= inference_interval:
        results = model(frame, conf=0.1)  # lower confidence threshold
        last_inference_time = current_time

        if results[0].boxes is not None and len(results[0].boxes) > 0:
            for box in results[0].boxes:
                class_id = int(box.cls[0])
                confidence = box.conf[0].item()
                label = f"{emotion_classes[class_id]} ({confidence:.2f})"
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
            last_label = label
        else:
            last_label = "No emotion detected"

    cv2.putText(frame, last_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("YOLOv11 Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 256x256 boring 0.51, neutral 0.15, disgust 0.13, suprised 0.08, sad 0.05, 8.2ms
Speed: 18.1ms preprocess, 8.2ms inference, 0.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 boring 0.86, sad 0.04, happy 0.04, disgust 0.01, suprised 0.01, 8.4ms
Speed: 5.8ms preprocess, 8.4ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 happy 0.46, suprised 0.26, sad 0.16, disgust 0.04, neutral 0.03, 9.0ms
Speed: 6.4ms preprocess, 9.0ms inference, 0.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 suprised 0.24, sad 0.24, neutral 0.19, disgust 0.09, angry 0.08, 7.6ms
Speed: 6.6ms preprocess, 7.6ms inference, 0.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 sad 0.34, neutral 0.18, suprised 0.16, boring 0.08, disgust 0.08, 8.5ms
Speed: 6.4ms preprocess, 8.5ms inference, 0.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 sad 0.31, suprised 0.23, neutral 0.14, happy 0.08, disgust 0.07, 9.1ms
Speed: 7.0ms preproc

In [1]:
# YOLO with harcascase face detection

import cv2
from ultralytics import YOLO

# Load Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load your YOLO classification model
model = YOLO("D:/RuhunaNew/Academic/Research/Facial_Recog_Repo/Group_50_Repo/Final_Best_Codes/Models/yolo11n-cls.pt")


# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)  # (x, y, w, h) for each face

    for (x, y, w, h) in faces:
        # Crop the face from the frame
        face_img = frame[y:y+h, x:x+w]

        # (Optional) Resize face_img to the input size expected by your YOLO model
        # face_img = cv2.resize(face_img, (224, 224))  # Adjust size as needed

        # Run YOLO classification on the cropped face
        results = model(face_img)

        # Visualize results (optional: you can draw on the original frame)
        annotated_face = results[0].plot()
        frame[y:y+h, x:x+w] = cv2.resize(annotated_face, (w, h))

        # Draw bounding box around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Show the frame with annotations
    cv2.imshow("Face Detection + YOLO Classification", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 256x256 sad 0.56, neutral 0.19, boring 0.15, happy 0.03, suprised 0.02, 8.0ms
Speed: 9.1ms preprocess, 8.0ms inference, 0.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 sad 0.57, neutral 0.22, angry 0.06, boring 0.05, happy 0.05, 8.8ms
Speed: 3.3ms preprocess, 8.8ms inference, 0.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 sad 0.49, neutral 0.38, happy 0.04, angry 0.03, boring 0.02, 6.7ms
Speed: 3.0ms preprocess, 6.7ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 sad 0.42, neutral 0.30, angry 0.12, happy 0.08, suprised 0.03, 7.9ms
Speed: 2.4ms preprocess, 7.9ms inference, 0.0ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 sad 0.45, neutral 0.27, angry 0.15, boring 0.04, disgust 0.03, 7.5ms
Speed: 3.0ms preprocess, 7.5ms inference, 0.1ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 sad 0.48, neutral 0.28, happy 0.10, angry 0.05, boring 0.03, 7.0ms
Speed: 2.3ms preprocess, 7.0ms inferenc

KeyboardInterrupt: 